In [1]:
import numpy as np
import pandas as pd

# Cargo la base que ya limpiamos y exportamos en el modelo 1
df = pd.read_csv('df_cleaned.csv')

# Me aseguro de que no haya precios nulos o no positivos
df = df[df['price'] > 0].copy()

# Target en log(precio) para estabilizar la varianza
df['log_price'] = np.log(df['price'])

# Tomo solo columnas numéricas (incluye dummies 0/1)
numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()

# Quito la columna de precio original y el log_price del set de features
numeric_cols.remove('price')
numeric_cols.remove('log_price')

# Matrices finales para el modelo
X = df[numeric_cols].values
y = df['log_price'].values

print("Shape X:", X.shape)
print("Shape y:", y.shape)
print("Número de features:", len(numeric_cols))


Shape X: (20827, 98)
Shape y: (20827,)
Número de features: 98
